<a href="https://colab.research.google.com/github/elangbijak4/Blockchain_Research/blob/main/Framework_Mini1_Infrastruktur_Blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import hashlib
import time
import json
import random
import os

In [2]:
class Block:
    def __init__(self, index, previous_hash, timestamp, data, hash):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.hash = hash

    @staticmethod
    def calculate_hash(index, previous_hash, timestamp, data):
        value = f"{index}{previous_hash}{timestamp}{data}"
        return hashlib.sha256(value.encode('utf-8')).hexdigest()

    @staticmethod
    def create_genesis_block():
        return Block(0, "0", int(time.time()), "Genesis Block", Block.calculate_hash(0, "0", int(time.time()), "Genesis Block"))

    def to_dict(self):
        return {
            "index": self.index,
            "previous_hash": self.previous_hash,
            "timestamp": self.timestamp,
            "data": self.data,
            "hash": self.hash
        }

In [3]:
class ProofOfElapsedTime:
    def __init__(self):
        self.wait_times = {}

    def wait_for_block(self, address):
        wait_time = random.uniform(0, 10)  # Simulating wait time
        self.wait_times[address] = wait_time
        time.sleep(wait_time)

    def select_validator(self):
        return min(self.wait_times, key=self.wait_times.get)

In [4]:
class ProofOfBurn:
    def __init__(self):
        self.burned_coins = {}  # To keep track of burned coins by address

    def burn_coins(self, address, amount):
        if address in self.burned_coins:
            self.burned_coins[address] += amount
        else:
            self.burned_coins[address] = amount

    def select_validator(self):
        total_burned = sum(self.burned_coins.values())
        pick = random.uniform(0, total_burned)
        current = 0
        for address, burned in self.burned_coins.items():
            current += burned
            if current > pick:
                return address

In [5]:
class ProofOfStake:
    def __init__(self):
        self.stakeholders = {}  # To keep track of stakeholder's stakes

    def add_stakeholder(self, address, stake):
        if address in self.stakeholders:
            self.stakeholders[address] += stake
        else:
            self.stakeholders[address] = stake

    def select_validator(self):
        total_stake = sum(self.stakeholders.values())
        pick = random.uniform(0, total_stake)
        current = 0
        for address, stake in self.stakeholders.items():
            current += stake
            if current > pick:
                return address

In [6]:
class Blockchain:
    def __init__(self, consensus_protocol, filename='blockchain.json'):
        self.filename = filename
        self.consensus_protocol = consensus_protocol
        self.chain = self.load_chain()

    def load_chain(self):
        if os.path.exists(self.filename):
            with open(self.filename, 'r') as file:
                data = json.load(file)
                return [Block(**block) for block in data]
        else:
            return [Block.create_genesis_block()]

    def save_chain(self):
        with open(self.filename, 'w') as file:
            json.dump([block.to_dict() for block in self.chain], file, indent=4)

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, data):
        latest_block = self.get_latest_block()
        index = latest_block.index + 1
        timestamp = int(time.time())
        previous_hash = latest_block.hash
        new_block = Block(index, previous_hash, timestamp, data, "")

        if isinstance(self.consensus_protocol, ProofOfBurn):
            validator = self.consensus_protocol.select_validator()
            print(f"Block #{index} created by {validator} using PoB")
        elif isinstance(self.consensus_protocol, ProofOfElapsedTime):
            for address in self.consensus_protocol.wait_times.keys():
                self.consensus_protocol.wait_for_block(address)
            validator = self.consensus_protocol.select_validator()
            print(f"Block #{index} created by {validator} using PoET")

        new_block.hash = Block.calculate_hash(new_block.index, new_block.previous_hash, new_block.timestamp, new_block.data)
        self.chain.append(new_block)
        self.save_chain()

    def to_json(self):
        self.save_chain()
        with open(self.filename, 'r') as file:
            return file.read()

In [7]:
class SmartContract:
    def __init__(self, code):
        self.code = code
        self.storage = {}

    def execute(self, blockchain, **kwargs):
        local_context = {
            'blockchain': blockchain,
            'storage': self.storage
        }
        local_context.update(kwargs)
        exec(self.code, {}, local_context)
        self.storage = local_context['storage']

In [8]:
class ERC721:
    def __init__(self):
        self.tokens = {}
        self.token_owners = {}
        self.token_metadata = {}
        self._token_owner = {}  # tokenId => owner
        self._owned_tokens = {}  # owner => list of tokenIds
        self._token_approvals = {}  # tokenId => approved address
        self._operator_approvals = {}  # owner => (operator => approved)

    def _mint(self, to, tokenId, metadata):
        if to in self._owned_tokens:
            self._owned_tokens[to].append(tokenId)
        else:
            self._owned_tokens[to] = [tokenId]
        self._token_owner[tokenId] = to
        self.token_metadata[tokenId] = metadata

    def _transfer(self, from_, to, tokenId):
        self._owned_tokens[from_].remove(tokenId)
        if to in self._owned_tokens:
            self._owned_tokens[to].append(tokenId)
        else:
            self._owned_tokens[to] = [tokenId]
        self._token_owner[tokenId] = to

    def mint(self, to, metadata):
        tokenId = hashlib.sha256(metadata.encode()).hexdigest()
        self._mint(to, tokenId, metadata)
        return tokenId

    def transfer(self, from_, to, tokenId):
        self._transfer(from_, to, tokenId)

In [9]:
class FungibleToken:
    def __init__(self):
        self.balances = {}
        self.total_supply = 0

    def mint(self, to, amount):
        if to in self.balances:
            self.balances[to] += amount
        else:
            self.balances[to] = amount
        self.total_supply += amount

    def transfer(self, from_, to, amount):
        if from_ not in self.balances or self.balances[from_] < amount:
            raise ValueError("Insufficient balance")
        if to in self.balances:
            self.balances[to] += amount
        else:
            self.balances[to] = amount
        self.balances[from_] -= amount

    def balance_of(self, address):
        return self.balances.get(address, 0)

In [20]:
class MyBlockchain(Blockchain):
    def __init__(self,pos):
        super().__init__(pos)
        self.smart_contracts = {}
        self.nft_contract = ERC721()
        self.fungible_token_contract = FungibleToken()
        self.pos = pos

    def store_smart_contract(self, code):
        latest_block = self.get_latest_block()
        index = latest_block.index + 1
        timestamp = int(time.time())
        previous_hash = latest_block.hash
        smart_contract = SmartContract(code)
        new_block = Block(index, previous_hash, timestamp, code, "")
        validator = self.pos.select_validator()
        print(f"Smart Contract Block #{index} created by {validator} using PoS")
        new_block.hash = Block.calculate_hash(new_block.index, new_block.previous_hash, new_block.timestamp, new_block.data)
        self.chain.append(new_block)
        block_hash = new_block.hash
        self.smart_contracts[block_hash] = smart_contract
        return block_hash

    def execute_smart_contract_from_block(self, block_hash, **kwargs):
        if block_hash in self.smart_contracts:
            smart_contract = self.smart_contracts[block_hash]
            smart_contract.execute(self, **kwargs)
        else:
            print("Smart contract not found in blockchain")

    def mint_nft(self, owner, metadata):
        tokenId = self.nft_contract.mint(owner, metadata)
        data = json.dumps({
            "type": "mint",
            "owner": owner,
            "tokenId": tokenId,
            "metadata": metadata
        })
        self.add_block(data)
        return self.get_latest_block().hash

    def transfer_nft(self, from_, to, tokenId):
        self.nft_contract.transfer(from_, to, tokenId)
        data = json.dumps({
            "type": "transfer",
            "from": from_,
            "to": to,
            "tokenId": tokenId
        })
        self.add_block(data)
        return self.get_latest_block().hash

    def mint_fungible_token(self, to, amount):
        self.fungible_token_contract.mint(to, amount)
        data = json.dumps({
            "type": "mint_fungible",
            "to": to,
            "amount": amount
        })
        self.add_block(data)
        return self.get_latest_block().hash

    def transfer_fungible_token(self, from_, to, amount):
        self.fungible_token_contract.transfer(from_, to, amount)
        data = json.dumps({
            "type": "transfer_fungible",
            "from": from_,
            "to": to,
            "amount": amount
        })
        self.add_block(data)
        return self.get_latest_block().hash

In [21]:
# Contoh Penggunaan
pos = ProofOfStake()
my_blockchain = MyBlockchain(pos)

In [22]:
# Add stakeholders using the ProofOfStake object
pos.add_stakeholder('Alice', 10)
pos.add_stakeholder('Bob', 20)
pos.add_stakeholder('Charlie', 30)

In [23]:
# Menambahkan blok biasa
my_blockchain.add_block('Block 1 Data')
my_blockchain.add_block('Block 2 Data')

In [24]:
# Menambahkan smart contract
smart_contract_code = """
def contract_function(storage):
    storage['counter'] = storage.get('counter', 0) + 1
    print(f"Counter value: {storage['counter']}")
contract_function(storage)
"""

In [25]:
block_hash = my_blockchain.store_smart_contract(smart_contract_code)
print(f"Smart contract stored in block with hash: {block_hash}")

Smart Contract Block #3 created by Bob using PoS
Smart contract stored in block with hash: 0ab93c15e23019f319660b1717b9dad7cfcd5b6dc9d919214e54daa25a2ad761


In [26]:
# Menjalankan smart contract dari blockchain
my_blockchain.execute_smart_contract_from_block(block_hash)

Counter value: 1


In [27]:
# Menjalankan kembali smart contract untuk melihat perubahan state
my_blockchain.execute_smart_contract_from_block(block_hash)

Counter value: 2


In [28]:
# Mencetak isi blockchain dalam format JSON
print("Blockchain content in JSON:")
print(my_blockchain.to_json())

Blockchain content in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1719854156,
        "data": "Genesis Block",
        "hash": "b799dde4c6ce0686ab2e2d4a6355a33af6abe276d309bbb740bd79de3695853b"
    },
    {
        "index": 1,
        "previous_hash": "b799dde4c6ce0686ab2e2d4a6355a33af6abe276d309bbb740bd79de3695853b",
        "timestamp": 1719854183,
        "data": "Block 1 Data",
        "hash": "c93e6374d931cf51a65ef9ede6876af01ff3a01a89093c1474da855a3bb9c1d2"
    },
    {
        "index": 2,
        "previous_hash": "c93e6374d931cf51a65ef9ede6876af01ff3a01a89093c1474da855a3bb9c1d2",
        "timestamp": 1719854183,
        "data": "Block 2 Data",
        "hash": "90a4188a7fc937f1a8f59283469090058bf8c3169be670df8c0e0fa5a35ab168"
    },
    {
        "index": 3,
        "previous_hash": "90a4188a7fc937f1a8f59283469090058bf8c3169be670df8c0e0fa5a35ab168",
        "timestamp": 1719854195,
        "data": "\ndef contract_function(storage):\n    st

In [29]:
# Membuat NFT baru
owner_address = "0xfeedfacefeedfacefeedfacefeedfacefeedface"
asset_uri = "https://example2.co.id"
mint_block_hash = my_blockchain.mint_nft(owner_address, asset_uri)
print(f"NFT minted in block with hash: {mint_block_hash}")

NFT minted in block with hash: 444a9c5c8753f82d72a29148f97a97dd8f1330a91baad8d44e33bb5bab7c1fb5


In [30]:
# Transfer NFT
new_owner_address = "0xbadc0ffeebadc0ffeebadc0ffeebadc0ffeebadc"
token_id = hashlib.sha256(asset_uri.encode()).hexdigest()
transfer_block_hash = my_blockchain.transfer_nft(owner_address, new_owner_address, token_id)
print(f"NFT transferred in block with hash: {transfer_block_hash}")

NFT transferred in block with hash: a9991e837578ea895e53f3dd9be77b68040607d9e802b57f16ee08c22b8b6e15


In [31]:
# Mencetak isi blockchain dalam format JSON
print("Blockchain content in JSON:")
print(my_blockchain.to_json())

Blockchain content in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1719854156,
        "data": "Genesis Block",
        "hash": "b799dde4c6ce0686ab2e2d4a6355a33af6abe276d309bbb740bd79de3695853b"
    },
    {
        "index": 1,
        "previous_hash": "b799dde4c6ce0686ab2e2d4a6355a33af6abe276d309bbb740bd79de3695853b",
        "timestamp": 1719854183,
        "data": "Block 1 Data",
        "hash": "c93e6374d931cf51a65ef9ede6876af01ff3a01a89093c1474da855a3bb9c1d2"
    },
    {
        "index": 2,
        "previous_hash": "c93e6374d931cf51a65ef9ede6876af01ff3a01a89093c1474da855a3bb9c1d2",
        "timestamp": 1719854183,
        "data": "Block 2 Data",
        "hash": "90a4188a7fc937f1a8f59283469090058bf8c3169be670df8c0e0fa5a35ab168"
    },
    {
        "index": 3,
        "previous_hash": "90a4188a7fc937f1a8f59283469090058bf8c3169be670df8c0e0fa5a35ab168",
        "timestamp": 1719854195,
        "data": "\ndef contract_function(storage):\n    st

In [32]:
# Demonstrasi Token Fungible (ERC20)
my_blockchain.mint_fungible_token("0xabc", 1000)
my_blockchain.mint_fungible_token("0xdef", 500)

'abff7a8e42c479709d2d799a18a4607fba1956b69508e3dcf45304ad00ffbe80'

In [33]:
print("Balance of 0xabc:", my_blockchain.fungible_token_contract.balance_of("0xabc"))
print("Balance of 0xdef:", my_blockchain.fungible_token_contract.balance_of("0xdef"))

Balance of 0xabc: 1000
Balance of 0xdef: 500


In [34]:
my_blockchain.transfer_fungible_token("0xabc", "0xdef", 200)

'be1b041ab1b3c60f48b566a390d1d3d9a8a8a68f4493e38d2bfc6ca5b03031a1'

In [35]:
print("Balance of 0xabc after transfer:", my_blockchain.fungible_token_contract.balance_of("0xabc"))
print("Balance of 0xdef after transfer:", my_blockchain.fungible_token_contract.balance_of("0xdef"))

Balance of 0xabc after transfer: 800
Balance of 0xdef after transfer: 700


In [36]:
# Mencetak isi blockchain dalam format JSON
print("Blockchain content in JSON:")
print(my_blockchain.to_json())

Blockchain content in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1719854156,
        "data": "Genesis Block",
        "hash": "b799dde4c6ce0686ab2e2d4a6355a33af6abe276d309bbb740bd79de3695853b"
    },
    {
        "index": 1,
        "previous_hash": "b799dde4c6ce0686ab2e2d4a6355a33af6abe276d309bbb740bd79de3695853b",
        "timestamp": 1719854183,
        "data": "Block 1 Data",
        "hash": "c93e6374d931cf51a65ef9ede6876af01ff3a01a89093c1474da855a3bb9c1d2"
    },
    {
        "index": 2,
        "previous_hash": "c93e6374d931cf51a65ef9ede6876af01ff3a01a89093c1474da855a3bb9c1d2",
        "timestamp": 1719854183,
        "data": "Block 2 Data",
        "hash": "90a4188a7fc937f1a8f59283469090058bf8c3169be670df8c0e0fa5a35ab168"
    },
    {
        "index": 3,
        "previous_hash": "90a4188a7fc937f1a8f59283469090058bf8c3169be670df8c0e0fa5a35ab168",
        "timestamp": 1719854195,
        "data": "\ndef contract_function(storage):\n    st

In [37]:
# Contoh Penggunaan
# Proof of Burn
pob = ProofOfBurn()
pob.burn_coins('Alice', 10)
pob.burn_coins('Bob', 20)
pob.burn_coins('Charlie', 30)

blockchain_pob = Blockchain(pob)
blockchain_pob.add_block('Block 1 Data')
blockchain_pob.add_block('Block 2 Data')

print("Blockchain content with PoB in JSON:")
print(blockchain_pob.to_json())

# Proof of Elapsed Time
poet = ProofOfElapsedTime()
poet.wait_times = {'Alice': 0, 'Bob': 0, 'Charlie': 0}

blockchain_poet = Blockchain(poet)
blockchain_poet.add_block('Block 1 Data')
blockchain_poet.add_block('Block 2 Data')

print("Blockchain content with PoET in JSON:")
print(blockchain_poet.to_json())

Block #9 created by Charlie using PoB
Block #10 created by Charlie using PoB
Blockchain content with PoB in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1719854156,
        "data": "Genesis Block",
        "hash": "b799dde4c6ce0686ab2e2d4a6355a33af6abe276d309bbb740bd79de3695853b"
    },
    {
        "index": 1,
        "previous_hash": "b799dde4c6ce0686ab2e2d4a6355a33af6abe276d309bbb740bd79de3695853b",
        "timestamp": 1719854183,
        "data": "Block 1 Data",
        "hash": "c93e6374d931cf51a65ef9ede6876af01ff3a01a89093c1474da855a3bb9c1d2"
    },
    {
        "index": 2,
        "previous_hash": "c93e6374d931cf51a65ef9ede6876af01ff3a01a89093c1474da855a3bb9c1d2",
        "timestamp": 1719854183,
        "data": "Block 2 Data",
        "hash": "90a4188a7fc937f1a8f59283469090058bf8c3169be670df8c0e0fa5a35ab168"
    },
    {
        "index": 3,
        "previous_hash": "90a4188a7fc937f1a8f59283469090058bf8c3169be670df8c0e0fa5a35ab168",
     